In [1]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
#####Returns: April 2nd to May 2nd#####

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1522627200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525219201:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [10]:
#####Factors: January to April#####
#####log_mkcap and coin_ratio computed averaging.
factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5

####regression against returns from April 2nd to May 2nd; loadings computed on April 1st.####
reg=linear_model.LinearRegression()
reg.fit(factors,R.T)
beta=reg.coef_
inter=reg.intercept_
inter=inter.reshape(-1,1)
total_beta=np.hstack((beta,inter)) 
total_factors=np.hstack((factors, np.ones(factors.shape[0]).reshape(-1,1)))
R_hat=reg.predict(factors) #predicted returns = (tot_fact).(tot_beta.T)
#####compute r_squared and F_scores#####
residuals=R.T-R_hat
R_mean=np.mean(R.T,axis=0) #daily average return
tss=np.sum( (R.T-R_mean)**2, axis=0 ) #daily total_sum_squares
rss=np.sum( (R.T-R_hat)**2, axis=0 ) #daily residual_sum_squares ~ var residuals
r2=1-rss/tss #daily R^2
cov_residuals=np.dot(residuals.T,residuals)/(factors.shape[0]-factors.shape[1])#daily
var_residuals=cov_residuals.diagonal() #daily unbiased variance of residuals
F=((tss-rss)/factors.shape[1])/var_residuals #daily F-scores
#####compute z_scores, using total_beta and total_factors for compactness#####
inv=np.linalg.inv(np.dot(total_factors.T,total_factors)) #usual (X^T.X)^{-1}
cov_beta=np.tensordot(inv,var_residuals, axes=0) #daily covariance of betas: inv*var(residual)
var_beta=cov_beta.diagonal() #daily variance of betas
std_beta=np.sqrt(var_beta) #daily standard errors
z_scores=total_beta/std_beta
df_list=[]
for t in range(30):
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df['factor_name']=['returns_variance', 'returns_strength', 'returns_high_low', 'turnover', 'average_log_mkcap', 'average_coin_ratio','1']
    df['beta']=total_beta[t]
    df['std_error']=std_beta[t]
    df['z_score']=z_scores[t]
    df1['r_squared']=np.array([r2[t]])
    df1['F_score']=np.array([F[t]])
    df_list.append([df,df1])
    
for x in df_list:
    for y in x:
        print(y)

          factor_name      beta  std_error   z_score
0    returns_variance  0.005673   0.005078  1.117177
1    returns_strength -0.008909   0.006503 -1.369943
2    returns_high_low -0.007724   0.008441 -0.915063
3            turnover  0.008477   0.003995  2.121834
4   average_log_mkcap  0.014429   0.006068  2.377955
5  average_coin_ratio -0.007560   0.004187 -1.805513
6                   1  0.025469   0.003956  6.438264
   r_squared   F_score
0   0.216551  3.777567
          factor_name      beta  std_error   z_score
0    returns_variance -0.001273   0.008117 -0.156793
1    returns_strength -0.008893   0.010396 -0.855381
2    returns_high_low  0.027772   0.013494  2.058096
3            turnover -0.005393   0.006386 -0.844455
4   average_log_mkcap  0.001225   0.009700  0.126265
5  average_coin_ratio  0.019901   0.006693  2.973272
6                   1  0.046658   0.006324  7.378179
   r_squared   F_score
0   0.351459  7.406265
          factor_name      beta  std_error    z_score
0    r

In [11]:
#####Factors: January to April#####
#####log_mkcap and coin_ratio computed exact on April 1st.

factors=[]
factor_names1_=['variance_']
factor_names1=['strength_', 'high_low_']
factor_names2=['turnover_']
factor_names2_=['log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1_:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            x=np.sqrt(x)
            sym_row.append(x)
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2_:
            with open(path+name+'exact'+'_'+sym+'_'+str(1522540800)+'.txt') as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+'exact'+'_'+base+'_'+str(1522540800)+'.txt') as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5

####regression against returns from April 2nd to May 2nd; loadings computed on April 1st.####
reg=linear_model.LinearRegression()
reg.fit(factors,R.T)
beta=reg.coef_
inter=reg.intercept_
inter=inter.reshape(-1,1)
total_beta=np.hstack((beta,inter)) 
total_factors=np.hstack((factors, np.ones(factors.shape[0]).reshape(-1,1)))
R_hat=reg.predict(factors) #predicted returns = (tot_fact).(tot_beta.T)
#####compute r_squared and F_scores#####
residuals=R.T-R_hat
R_mean=np.mean(R.T,axis=0) #daily average return
tss=np.sum( (R.T-R_mean)**2, axis=0 ) #daily total_sum_squares
rss=np.sum( (R.T-R_hat)**2, axis=0 ) #daily residual_sum_squares ~ var residuals
r2=1-rss/tss #daily R^2
cov_residuals=np.dot(residuals.T,residuals)/(factors.shape[0]-factors.shape[1])#daily
var_residuals=cov_residuals.diagonal() #daily unbiased variance of residuals
F=((tss-rss)/factors.shape[1])/var_residuals #daily F-scores
#####compute z_scores, using total_beta and total_factors for compactness#####
inv=np.linalg.inv(np.dot(total_factors.T,total_factors)) #usual (X^T.X)^{-1}
cov_beta=np.tensordot(inv,var_residuals, axes=0) #daily covariance of betas: inv*var(residual)
var_beta=cov_beta.diagonal() #daily variance of betas
std_beta=np.sqrt(var_beta) #daily standard errors
z_scores=total_beta/std_beta
df_list=[]
for t in range(30):
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df['factor_name']=['returns_std', 'returns_strength', 'returns_high_low', 'turnover', 'log_mkcap', 'coin_ratio','1']
    df['beta']=total_beta[t]
    df['std_error']=std_beta[t]
    df['z_score']=z_scores[t]
    df1['r_squared']=np.array([r2[t]])
    df1['F_score']=np.array([F[t]])
    df_list.append([df,df1])
for x in df_list:
    for y in x:
        print(y)

        factor_name      beta  std_error   z_score
0       returns_std  0.011903   0.006302  1.888699
1  returns_strength -0.011805   0.006507 -1.814194
2  returns_high_low -0.005224   0.008604 -0.607072
3          turnover  0.006631   0.004058  1.634095
4         log_mkcap  0.001561   0.004212  0.370661
5        coin_ratio -0.006118   0.004259 -1.436581
6                 1  0.025469   0.004023  6.331031
   r_squared   F_score
0   0.189787  3.201323
        factor_name      beta  std_error   z_score
0       returns_std -0.000292   0.009871 -0.029623
1  returns_strength -0.010753   0.010192 -1.055089
2  returns_high_low  0.027105   0.013476  2.011339
3          turnover -0.005698   0.006356 -0.896512
4         log_mkcap -0.005595   0.006597 -0.848069
5        coin_ratio  0.021271   0.006670  3.188937
6                 1  0.046658   0.006301  7.405401
   r_squared   F_score
0   0.356218  7.562052
        factor_name      beta  std_error    z_score
0       returns_std -0.013671   0.006889

   r_squared   F_score
0   0.183074  3.062709
        factor_name      beta  std_error   z_score
0       returns_std  0.000307   0.016561  0.018517
1  returns_strength  0.037277   0.017099  2.180023
2  returns_high_low -0.028702   0.022610 -1.269409
3          turnover -0.006214   0.010663 -0.582712
4         log_mkcap -0.003395   0.011068 -0.306701
5        coin_ratio -0.001715   0.011191 -0.153273
6                 1 -0.050455   0.010571 -4.772949
   r_squared   F_score
0   0.301297  5.893384
        factor_name      beta  std_error   z_score
0       returns_std  0.003832   0.008279  0.462908
1  returns_strength -0.019758   0.008548 -2.311519
2  returns_high_low -0.008682   0.011303 -0.768157
3          turnover  0.006410   0.005331  1.202504
4         log_mkcap  0.001094   0.005533  0.197643
5        coin_ratio -0.001698   0.005594 -0.303439
6                 1  0.046759   0.005284  8.848555
   r_squared   F_score
0   0.110334  1.694898
        factor_name      beta  std_error   z_s

In [5]:
#regression against returns on April 1st only.
t=1522540800
ret=[]
for base in coins:
    for quote in quotes:
        returns=upload_log_return(t, base, quote, 86400)
        if np.isnan(returns)==True:
            returns=0
        ret.append(returns)
ret=np.array(ret)
reg=linear_model.LinearRegression()
reg.fit(factors,ret.T)
beta=reg.coef_
inter=reg.intercept_
ret_hat=reg.predict(factors)
rss=np.linalg.norm(ret-ret_hat)**2
var_error=rss/(factors.shape[0]-factors.shape[1])
tss=np.linalg.norm(ret-np.mean(ret))**2
r_square=1-rss/tss
F_score=(tss-rss)/(factors.shape[1]*var_error)
cov_beta=np.linalg.inv(np.dot(factors.T,factors))*var_error
std_errors=np.sqrt(cov_beta.diagonal())
t_stats=beta/std_errors

df=pd.DataFrame()
df['factor_name']=['returns_variance', 'returns_strength', 'returns_high_low', 'turnover', 'log_mkcap', 'coin_ratio']
df['beta']=beta
df['std_error']=std_errors
df['z_score']=t_stats
df1=pd.DataFrame()
df1['r_squared']=np.array([r_square])
df1['F_score']=np.array([F_score])
print('Regression on April 1st returns.')
print('Definition of factors, for each coin pair:')
print('*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.')
print('*returns_strength=sum_t( log(1+return_t) ) over returns as above.')
print('*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.')
print('*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.')
print('*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.')
print('*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.')
print('Factor loadings have been scaled by z-score.')
print('')
print(df)
print(df1)

Regression on April 1st returns.
Definition of factors, for each coin pair:
*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.
*returns_strength=sum_t( log(1+return_t) ) over returns as above.
*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.
*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.
*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.
*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.
Factor loadings have been scaled by z-score.

        factor_name      beta  std_error   z_score
0  returns_variance -0.001987   0.004414 -0.450239
1  returns_strength  0.010160   0.005653  1.797171
2  returns_high_low  0.005005   0.007338  0.682049
3          turnover -0.000110   0.003473 -0.031699
4         log_mkcap -0.010219   0.005275 -1.937250
5        coin_ratio  0.005506   0.003640  1.512765
   r_squared   F_score
0   0.190144  3.

In [7]:
####regression against returns from April 2nd to May 2nd; loadings computed on April 1st.####
reg=linear_model.LinearRegression()
reg.fit(factors,R.T)
beta=reg.coef_
inter=reg.intercept_
inter=inter.reshape(-1,1)
total_beta=np.hstack((beta,inter)) 
total_factors=np.hstack((factors, np.ones(factors.shape[0]).reshape(-1,1)))
R_hat=reg.predict(factors) #predicted returns = (tot_fact).(tot_beta.T)
#####compute r_squared and F_scores#####
residuals=R.T-R_hat
R_mean=np.mean(R.T,axis=0) #daily average return
tss=np.sum( (R.T-R_mean)**2, axis=0 ) #daily total_sum_squares
rss=np.sum( (R.T-R_hat)**2, axis=0 ) #daily residual_sum_squares ~ var residuals
r2=1-rss/tss #daily R^2
cov_residuals=np.dot(residuals.T,residuals)/(factors.shape[0]-factors.shape[1])#daily
var_residuals=cov_residuals.diagonal() #daily unbiased variance of residuals
F=((tss-rss)/factors.shape[1])/var_residuals #daily F-scores
#####compute z_scores, using total_beta and total_factors for compactness#####
inv=np.linalg.inv(np.dot(total_factors.T,total_factors)) #usual (X^T.X)^{-1}
cov_beta=np.tensordot(inv,var_residuals, axes=0) #daily covariance of betas: inv*var(residual)
var_beta=cov_beta.diagonal() #daily variance of betas
std_beta=np.sqrt(var_beta) #daily standard errors
z_scores=total_beta/std_beta
df_list=[]
for t in range(30):
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df['factor_name']=['returns_variance', 'returns_strength', 'returns_high_low', 'turnover', 'log_mkcap', 'coin_ratio','1']
    df['beta']=total_beta[t]
    df['std_error']=std_beta[t]
    df['z_score']=z_scores[t]
    df1['r_squared']=np.array([r2[t]])
    df1['F_score']=np.array([F[t]])
    df_list.append([df,df1])



In [8]:
print('Definition of factors, for each coin pair:')
print('*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.')
print('*returns_strength=sum_t( log(1+return_t) ) over returns as above.')
print('*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.')
print('*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.')
print('*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.')
print('*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.')
print('Factor loadings have been scaled by z-score.')
print('')
for y in df_list:    
    for x in y:
        print(x)

Definition of factors, for each coin pair:
*returns_variance = var(returns_t), over weekly returns Jan 7th-Apr 1st.
*returns_strength=sum_t( log(1+return_t) ) over returns as above.
*returns_high_low=log( max_t(return_t)/min_t(return_t) ) over returns as above.
*turnover=(total traded volume)/(average coin supply) over trades Jan 7th-Apr 1st.
*log_mkcap=average_t( log(coin_supply_t*price_t) ) over weekly supply and price Jan 7th-Apr 1st.
*coin_ratio=average_t(coin_supply_t/coin_supply_ever) over weekly supply as above.
Factor loadings have been scaled by z-score.

        factor_name      beta  std_error   z_score
0  returns_variance  0.005673   0.005078  1.117177
1  returns_strength -0.008909   0.006503 -1.369943
2  returns_high_low -0.007724   0.008441 -0.915063
3          turnover  0.008477   0.003995  2.121834
4         log_mkcap  0.014429   0.006068  2.377955
5        coin_ratio -0.007560   0.004187 -1.805513
6                 1  0.025469   0.003956  6.438264
   r_squared   F_scor

In [ ]:
#####February to May#####

from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1517702400
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525564801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1517702400)+'_'+str(1525564800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1517702400)+'_'+str(1525564800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5
#X_with_ones = np.hstack((np.ones((factors.shape[0], 1)), factors))

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
#reg.fit pre-processes factors - centers along axis=0
#adding a colums of ones to factors makes no difference, there will be intercept anyway.
beta_=reg.coef_
inter_=reg.intercept_

R_hat1=reg.predict(factors)
R_hat=np.dot(factors,beta_.T)+inter_
print('r2 using daily mean =',reg.score(factors,R.T), r2_score(R.T,R_hat1,multioutput='variance_weighted'))
print('r2 using total mean =',r2_score(R.T,R_hat),)
#reg.score computes the mean along axis=0
#r2_score computes the mean over full matrix


In [ ]:
#factors must have extra column of ones for this to work.
factors=np.hstack((np.ones((factors.shape[0], 1)), factors))
beta=np.linalg.lstsq(factors, R.T,rcond=None)
gamma=beta[0]
R_hat_np=np.dot(factors,gamma)
residuals=np.sum( (R.T-R_hat_np)**2)
#average returns over days, for each coin
mean=np.mean(R, axis=0)
sum_squares=np.sum((R-mean.T)**2)
r2_coin=1-residuals/sum_squares
#average returns over coins, for each day.
mean_bs=np.mean(R, axis=1)
sum_squares_bs=np.sum((R.T-mean_bs)**2)
r2_daily=1-residuals/sum_squares_bs
print('r2 using coin mean =',r2_coin)
print('r2 using daily mean =',r2_daily)
#just average over days and coins
print('r2 using total mean =',1-residuals/np.linalg.norm(R.T-np.mean(R))**2)

In [ ]:
######################################################
######################################################
######################################################

In [ ]:
#regression against each one single time, BTC/USD
factor_test=factors
r=[]
for i in range(8):
    R_test=R[i]
    beta_test=np.linalg.lstsq(factor_test,R_test.T, rcond=None)
    S=np.dot(factors,beta_test[0])
    res_test=np.linalg.norm(R_test-S)**2
    r.append(1-res_test/np.linalg.norm(R_test)**2)
r=np.array(r)
print(np.max(r), np.min(r))

In [ ]:
reg=linear_model.LinearRegression()
for i in range(6):
    fac=factors.T[i].reshape(-1,1)
    ret=R[8].T
    #print(fac.shape)
    reg.fit(fac,ret)
    temp_beta=reg.coef_[0]
    temp_inter=reg.intercept_
    print(temp_beta, temp_inter)
    print(reg.score(fac,ret))
    plt.plot(fac, ret,'o')
    plt.plot(fac, temp_beta*fac + temp_inter, 'r', label='Fitted line')
    plt.show()

In [ ]:
for t in range(88):
    print(coins[int(t/2)],quotes[t%2])
    plt.plot(range(len(R)), R.T[t])
    plt.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf